# Tâche 4 : Question-réponse avec affinage par instructions du modèle GPT‑2 pré-entraîné sur Sherlock Holmes

**Objectifs**

Évaluer la qualité des réponses d’un modèle de langage **pré‑entraîné** (celui de la tâche 3) et affiner par instructions (cette tâche).
Dans ce *notebook*, vous faites le post-entraînement du modèle avec des instructions générales indiquant au modèle comment accomplir des tâches simples. Comme pour les tâches 2 et 3, la démarche de test est de construire un *prompt*, de générer des réponses, et d'évaluer qualitativement la pertinence des résultats. Plusieurs de ces fonctions sont rendues disponibles.

**Objectifs d’apprentissage**
1. Faire le post-entraînement d'un modèle pré‑entraîné avec l'affinage par instructions (*instruction tuning*).
2. Comprendre et expliquer les **limites et apports de l'affinage par instructions** d'un modèle.

Tout comme dans les tâches 2 et 3, les **questions** pour évaluer le modèle vous sont fournies. Le fichier d'**instructions** pour l'affinage du modèle est également fourni. Vous devez comprendre le format des questions chargées en mémoire. Il est également important de prendre connaissance de la nature des instructions utilisées pour l'affinage.

> Il est recommandé de faire ce travail pratique en utilisant une carte graphique GPU compatible avec HuggingFace/Pytorch.
> Si votre machine n’en possède pas, vous pouvez utiliser **Google Colab** pour exécuter le *notebook* dans le cloud.

Si nécessaire, installer les *packages* suivant. Si vous exécutez sur Code Colab, ces *packages* devraient déjà être installés.

In [1]:
#!pip install datasets
#!pip install accelerate
#!pip install 'transformers[torch]'
#!pip3 install torch torchvision

In [2]:
batch_size = 5 # il est possible d'ajuster la taille de batch. Les valeurs actuelles utilisent environ 10 Gb
max_length = 256 # on réduit le contexte pour sauver du temps, nos exemples ne nécesside pas un plus grand contexte

In [3]:
from datasets import Dataset
from transformers import pipeline, Trainer
import json
import os

In [4]:
try:
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False

#Mount a google drive folder to save things
if IN_COLAB:
  drive.mount('/content/drive')
  folders_to_mount = ["nlp_tp2_models", "results"]
  for folder in folders_to_mount:
    source = f'/content/drive/MyDrive/uni/nlp/{folder}'
    shortcut = f'/content/{folder}'
    print(f"Mounting {source} to {shortcut}")
    os.symlink(source, shortcut)

repo_url = "https://github.com/XavyShmore/tp2_nlp.git"
if IN_COLAB:
  !git clone {repo_url}
  !cp -r ./tp2_nlp/data .
  pass

Mounted at /content/drive
Mounting /content/drive/MyDrive/uni/nlp/nlp_tp2_models to /content/nlp_tp2_models
Mounting /content/drive/MyDrive/uni/nlp/results to /content/results
Cloning into 'tp2_nlp'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 49 (delta 26), reused 10 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (49/49), 2.48 MiB | 8.48 MiB/s, done.
Resolving deltas: 100% (26/26), done.


## 1. Chargement du modèle Hugging Face et du tokenizer (à compléter)

Complétez le corps de la fonction `load_model(model_path)` afin qu’elle :

- charge le **tokenizer** et le **modèle** Hugging Face à partir du chemin `model_path`.
- **retourne** le tokenizer comme **première valeur de retour** et le modèle comme **seconde valeur de retour**.

On ajoute également des fonctions pour monter les questions en mémoire et pour sauvegarder les réponses dans un fichier.

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
def load_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)

    return tokenizer, model

In [6]:
def load_entries(path):
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    if not isinstance(data, list):
        raise ValueError(f"Question file must contain a list of objects. Got: {type(data)}")
    return data

def save_answers(questions_answers, output_dir, out_file_name, display=True):
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, out_file_name), "w", encoding="utf-8") as out:
        for index, question, answer, expected_answer in questions_answers:
            out.write(f"Q: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}\n")
            if display:
                print(f"Q{index}: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}")

## 2. Fonctions de test question-réponse  (à compléter)

La fonction **test_on_questions** est utilisée pour parcourir **toutes les entrées** du fichier de questions afin de produire des réponses générées par le modèle.

La génération d'une réponse à une question implique les étapes suivantes (fonction **process_entry** à compléter) :
* Construire un prompt à l’aide de la fonction **alpaca_build_prompt** (rendu disponible dans la prochaine section)
* Utiliser le modèle (via un pipeline de génération de texte) pour générer une réponse à une question
* Retourner la réponse générée par le modèle.  

Points importants à souligner:
* La fonction *process_entry* doit retourner uniquement la réponse générée par le modèle (sans le prompt).
* Il est de votre responsabililté de choisir **les paramètres** du générateur (max_new_tokens, do_sample, temperature, top_k ou top_p). Décrivez ceux que vous avez retenus.

> Afin de simplifier le travail, nous avons choisi de ne pas utiliser de *batchs* dans la fonction qui teste les questions.
> Vous n'avez pas à prendre en compte le *warning* qui suggère d'utiliser des *datasets*.

Description des paramètres de génération:
(à compléter...)

In [58]:
def process_entry(entry, prompt_builder, generator):
    prompt = prompt_builder(entry)
    generation_output = generator(
        prompt,
        max_new_tokens=20, # limit the length of the generated answer
        do_sample=False,    # enable sampling for more varied responses
        temperature=0.4,   # control the randomness of the output
        top_k=50,          # consider only the top k most likely next tokens
        num_return_sequences=1
    )
    # The generated_text includes the prompt, so we need to extract only the answer
    generated_text = generation_output[0]['generated_text']
    answer = generated_text[len(prompt):].strip()
    return answer

In [100]:
#Ajouter cela avant la question afin pouvoir bien comparer avec les modèles précédent qui avaient cela dans leur prompt.
pre_prompt = "Answer the question "

In [93]:
def test_on_questions(prompt_builder, model_path, question_file, out_file_name, output_dir="results"):
    entries = load_entries(question_file)
    tokenizer, model = load_model(model_path)
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    results = []
    for i, entry in enumerate(entries):
        answer = process_entry({"instruction": pre_prompt + entry["question"]}, prompt_builder, generator)
        question = entry.get("question", "")
        expected_answer = entry.get("answer", "")
        results.append((i, question, answer, expected_answer))
    save_answers(results, output_dir, out_file_name, display=True)
    return results

## 3. Préparation des données et des prompts pour l'affinage du modèle

Le code suivant prépare les ressources nécessaires pour l'affinage du modèle GPT2 pré-entraîné dans la tâche 3 de ce travail.

Les étapes sont :
* Télécharger le fichier de données Alpaca, le jeu d'instructions utilisé pour l'affinage du modèle . Afin de limiter le temps d'entraînement, on retient seulement les 5000 premières instructions de ce *dataset*. Vous pouvez modifier ce nombre si vous le souhaitez.
* Générer un prompt spécifique à Alpaca.

On rend disponible tout ce qui est nécessaire pour ces 2 étapes.

In [9]:
import urllib.request

alpaca_url = "https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/refs/heads/main/alpaca_data.json"

def load_or_download_instruct_dataset_file(data_url, file_path, count=-1):
    with urllib.request.urlopen(data_url) as response:
        raw_data = response.read().decode("utf-8")
        data = json.loads(raw_data)
    if count > 0 and count <= len(data):
        data = data[:count]
    with open(file_path, "w", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=2)

instructions_fn = "data/alpaca_data.json"  # Fichier où sont enregistrées les instructions d'affinage du modèle
nb_instructions = 5000  # Ce nombre peut-être modifié
load_or_download_instruct_dataset_file(data_url=alpaca_url, count=nb_instructions, file_path=instructions_fn)

In [10]:
def alpaca_build_prompt(ex):
    instruction = ex.get("instruction", "")
    input = ex.get("input", "").strip()
    output = ex.get("output", "").strip()
    header = "Below is an instruction that describes a task"
    if input:
        header += ", paired with an input"
    return (
        f"{header}.\n"
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{instruction}\n\n"
        + (f"### Input:\n{input}\n\n" if input else "")
        + f"### Response:\n{output}"
    )

## 4. Affinage du modèle (à compléter)

Complétez le code suivant pour affiner le modèle GPT2 préentraîné et sauvegardé dans la tâche 3 de ce travail.

Les étapes à suivre sont de :
* Monter en mémoire le modèle pré-entraîné à la tâche 3 et son tokeniseur
* Monter le jeu d'instructions pour l'affinage du modèle et créer un *dataset* avec ces données.
* Tokeniser ce *dataset* d'instructions
* Faire l'entraînement du modèle sur le *dataset* avec la classe ***Trainer*** de Hugging Face
* Faire la sauvegarde du nouveau modèle dans un répertoire (voir *model_path*)

In [69]:
model_name = "./nlp_tp2_models/gpt2/gpt2-sherlock-lm" # Répertoire du modèle construit durant la tâche 3
tokenizer, model = load_model(model_name)

Création du *dataset* d'entraînement.

In [70]:
instructions_fn = "data/alpaca_data.json"  # Fichier qui contient les instructions d'affinage
dataset = Dataset.from_json(instructions_fn)

def create_prompt(example):
  example["text"] = alpaca_build_prompt(example)
  return example

txt_dataset = dataset.map(create_prompt)

Tokénisation du *dataset* d'entraînement.

In [71]:
def tokenize_example(example):
    return tokenizer(example["text"], truncation=True)

tokenized_dataset = txt_dataset.map(tokenize_example)
print(tokenized_dataset[2])

{'instruction': 'Describe the structure of an atom.', 'input': '', 'output': 'An atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.', 'text': 'Below is an instruction that describes a task.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nDescribe the structure of an atom.\n\n### Response:\nAn atom is made up of a nucleus, which contains protons and neutrons, surrounded by electrons that travel in orbits around the nucleus. The protons and neutrons have a positive charge, while the electrons have a negative charge, resulting in an overall neutral atom. The number of each particle determines the atomic number and the type of atom.', 'input_ids': [21106, 318, 281, 12064

In [72]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./nlp_tp2_models/gpt2/gpt2-sherlock-lm-instruct/checkpoints",
    overwrite_output_dir=True,

    num_train_epochs=1,

    # 2. BATCH SIZE: Small dataset = requires stability.
    # We use gradient accumulation to simulate a larger batch size (e.g., 32)
    # while keeping memory usage low.
    per_device_train_batch_size=12,
    gradient_accumulation_steps=3, # Effective batch size = 4 * 8 = 32

    # 3. LEARNING RATE: The most critical part.
    # Standard is 5e-5. Since your data is small, sticking to the lower end
    # prevents destroying the pre-trained knowledge.
    learning_rate=1e-5,
    weight_decay=0.01,

    # 4. SCHEDULER: vital for small data
    warmup_steps=50, # Warm up quickly (roughly 1 epoch worth of steps)
    lr_scheduler_type="cosine", # Smooth decay is better than linear for language

    # 5. LOGGING
    logging_steps=10,
    save_steps=50,
    fp16=True, # Use mixed precision if on GPU (much faster)
    report_to="none"
)

Ajouter dans ces cellules tout le code dont vous avez besoin pour faire l'affinage du modèle.

In [73]:
instruct_model_path = "./nlp_tp2_models/gpt2/gpt2-sherlock-lm-instruct" # Répertoire où sauvegarder le nouveau modèle et le tokenizer

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    )

In [74]:
trainer.train()

Step,Training Loss
10,3.477000
20,3.074200
30,2.622500
40,2.083500
50,1.761200
60,1.599600
70,1.562600
80,1.541200
90,1.552200
100,1.494300


TrainOutput(global_step=139, training_loss=1.9138462286201312, metrics={'train_runtime': 204.6475, 'train_samples_per_second': 24.432, 'train_steps_per_second': 0.679, 'total_flos': 2028267805900800.0, 'train_loss': 1.9138462286201312, 'epoch': 1.0})

Pour conclure cette section, sauvegardez le nouveau modèle et le tokenizer.

In [75]:
trainer.save_model(instruct_model_path)
tokenizer.save_pretrained(instruct_model_path)

('./nlp_tp2_models/gpt2/gpt2-sherlock-lm-instruct/tokenizer_config.json',
 './nlp_tp2_models/gpt2/gpt2-sherlock-lm-instruct/special_tokens_map.json',
 './nlp_tp2_models/gpt2/gpt2-sherlock-lm-instruct/vocab.json',
 './nlp_tp2_models/gpt2/gpt2-sherlock-lm-instruct/merges.txt',
 './nlp_tp2_models/gpt2/gpt2-sherlock-lm-instruct/added_tokens.json',
 './nlp_tp2_models/gpt2/gpt2-sherlock-lm-instruct/tokenizer.json')

## 5. Génération des réponses pour les questions de Sherlock Holmes

Exécutez la cellule suivante pour générer, avec le modèle affiné, les réponses aux questions de test.
Le temps d’exécution devrait se situer entre **5 et 10 minutes** si vous utilisez **Google Colab** avec un GPU.

Note : N'oubliez pas d'ajouter le fichier de réponses générées par le modèle (voir *out_file_name*) dans votre remise du travail.

In [101]:
questions_fn = "data/questions_sherlock.json"
out_file_name = "instruct_gpt2_answers.txt"

results = test_on_questions(prompt_builder=alpaca_build_prompt, model_path=instruct_model_path, question_file=questions_fn, out_file_name=out_file_name)

Device set to use cuda:0


Q0: Where do Sherlock Holmes and Dr. Watson live?
A: Sherlock Holmes and Dr. Watson live in the London suburb of Brixton. They are both doctors
Expected:221B Baker Street, London.
------------------------------------------------------------
Q1: Who is Sherlock Holmes' loyal friend and chronicler?
A: Sherlock Holmes' loyal friend and chronicler is the man who has been with him since the beginning
Expected:Dr. John H. Watson.
------------------------------------------------------------
Q2: Who is considered 'The Woman' by Sherlock Holmes?
A: The Woman is the most popular woman in the world. She is the most beautiful woman in the world
Expected:Irene Adler.
------------------------------------------------------------
Q3: Which story features the Red-Headed League?
A: The Red-Headed League is a fictional organization that is based in the fictional city of New York
Expected:The Adventure of the Red-Headed League.
------------------------------------------------------------
Q4: What is the p

 ## 6. Analyse des résultats

 ### 6.1 Évaluation quantitative (à compléter)

In [65]:
import string
import re
from collections import Counter

def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

def white_space_fix(text):
    return ' '.join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [66]:
def evaluate_f1(ground_truth, prediction):
    """Normalise les 2 textes, trouve ce qu'il y a en commun et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0, 0.0, 0.0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return precision, recall, f1

In [67]:
def evaluation_generation(results):
    eval = {"precision":0, "recall":0, "f1":0}
    for i, question, answer, expected_answer in results:
        precision, recall, f1 = evaluate_f1(expected_answer, answer)
        eval["precision"] += precision
        eval["recall"] += recall
        eval["f1"] += f1

    eval["precision"] /= len(results)
    eval["recall"] /= len(results)
    eval["f1"] /= len(results)

    return eval

In [102]:
eval = evaluation_generation(results)
print(eval)

{'precision': 0.0247940637793579, 'recall': 0.17952380952380953, 'f1': 0.04188327101175398}


**Question :** Que pensez-vous de cette évaluation ?

Les résultats sembles s'être fortement dégradé depuis la dernière étape. La précision tombe à 0.02 qui est pire que le résultat pour le premier modèle.

Le recall est à 0.179 ce qui est mieu que à la tâche 2 mais pire qu'à la tâche 3.

Cela donne un score f1 moyen qui est bien pire qu'avec le premier modèle. Je ne sait pas vraiment comment interpréter ces chiffres plus que cela. Je suis un peu déçu de ce résultat.

### 6.2 Analyse qualitative (à faire)

Faites l'analyse des réponses de ce modèle. Présentez vos observations par rapport aux réponses obtenus des modèles des tâches 2 et 3.

Expliquez ce que vous retenez des 3 dernières tâches sur le pré-entraînement et le post-entraînement du modèle GPT-2.

Vous pouvez ajouter des cellules au besoin.

En regardant les prédictions, il est possible d'un peu mieux comprendre ce qu'ils se passe.

Le fine-tuning avec des intrucctions semble avoir appris au modèle à répondre par des phrases complètes. Ainsi, si les réponses sont plus longues la précision en souffre car elle est calculé en comparant le nombre de mots corrects au nombre de mots dans la prédiction. Le reccal avantagé par les plus longue réponse.

Pourtant, l'origine du problème ne m'est pas claire.

Le modèle a-t-il été trop entrainé sur les instructions? Cela ferait est sorte qu'il tente de répondre par des réponse élaboré avec des phrases complètes comme dans le jeu d'entrainement.

Ou bien le modèle n'a pas été assez entrainé sur le instructions et ainsi, il a de la misère à respecter la consigne de faire un réponse courte.

Il serait aussi possible que j'ai trpo entrainé à la tâche 3 et ainsi écrasé des poids nécessaire pour bien comprendre les instructions.

Mais ils serait également possible qu'il n'ait pas été suffisament entrainé à l'étape précédente. Cela expliquerait des halucinations comme:

>**Q:** What was the profession of Dr. John Watson?
>
>**A:** Dr. John Watson was a British scientist and philosopher who was the founder of the field of artificial intelligence
>
>**Expected:** Doctor.

Dans tous les cas j'en retien qu'à toutes les étapes d'entrainement, la structure du text d'entrainement est très vite apprise. En effet, lors du pré entrainement, les guillements très présent dans le livre on faire leur apparition dans les prédictions. Également, les prédictions généré après l'entrainnement sur les instruction. on tout aussi vite perdu ces guillements mais ce sont mise à avoir la forme de vrais réponse.

Malgré tout l'information semble bien plus difficile à transmettre. Il y a un équilibre difficile entre écraser les poids précédents et capacitées pertinentes à chaque étape.

